In [75]:
import pandas as pd
import json

In [76]:
df = pd.read_excel('sheet.xlsx')

In [77]:
df

,Collection Title,Migrated?,Name of Resource,Sinequa Config Guess,Sinequa Config Actual,Connector Type,Assignee,URL_original,URL_sinequa,turned_on,...,UrlStayInside,UrlLinkExcluded,UrlDownloadExcluded,UrlFollowExcluded,UrlIndexExcluded,UrlIndexIncluded,IncludedExtensions,ExcludedExtensions,ExcludedFilenames,Cleaned
0,NaN,NaN,NAVO Registry,NaN,NaN,NaN,NaN,Use custom connector built by team,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Coordinate Calculator,NaN,NaN,NaN,NaN,http://ned.ipac.caltech.edu/coordinate_calculator,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Extinction Calculator,NaN,NaN,NaN,NaN,http://ned.ipac.caltech.edu/extinction_calculator,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Velocity Calculator,NaN,NaN,NaN,NaN,http://ned.ipac.caltech.edu/velocity_calculator,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,High Level Science Products,NaN,NaN,NaN,NaN,https://archive.stsci.edu/hlsp/,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,NaN,NaN,PDS Subset Tool Website,PDS_Subset_Tool_Website,NaN,NaN,NaN,NaN,https://naif.jpl.nasa.gov/cgi-bin/subsetds.pl?...,0.0,...,1.0,NaN,NaN,NaN,NaN,https://naif.jpl.nasa.gov/cgi-bin/subsetds.pl?...,htm;html,NaN,NaN,NaN
277,NaN,NaN,PDS Users Guides Website,PDS_Users_Guides_Website,NaN,NaN,NaN,NaN,https://pds-imaging.jpl.nasa.gov/software/,1.0,...,1.0,NaN,NaN,NaN,NaN,NaN,htm;html,NaN,NaN,NaN
278,NaN,NaN,PDS USGS Pilot Website,PDS_USGS_Pilot_Website,NaN,NaN,NaN,NaN,https://pilot.wr.usgs.gov/,1.0,...,1.0,NaN,NaN,NaN,NaN,https://pilot.wr.usgs.gov/,htm;html,NaN,NaN,NaN
279,NaN,NaN,PDS Web Chronos Website,PDS_Web_Chronos_Website,NaN,NaN,NaN,NaN,https://naif.jpl.nasa.gov/cgi-bin/chronos.pl?s...,0.0,...,1.0,NaN,NaN,NaN,NaN,https://naif.jpl.nasa.gov/cgi-bin/chronos.pl?s...,htm;html,NaN,NaN,NaN


In [78]:
columns_to_bring_in = {
    'Name of Resource': 'name',
    'Sinequa Config Guess': 'config_folder',
    'URL_original': 'url',
    'turned_on': 'turned_on',
    'Division': 'division',
    'Source': 'source',
    'TreeRoot': 'tree_root',
    'Frequency of Indexing/Updates': 'update_frequency',
    'Action': 'delete',
    'Audit Hierarchy (Matches hierarchy in spreedsheet - Yes / No) - SDE prod': 'audit_hierarchy',
    'Audit URL - SDE prod': 'audit_url',
    'Audit Mapping - SDE prod': 'audit_mapping',
    'Audit Label - SDE prod': 'audit_label',
    'Audit Query - SDE prod': 'audit_query',
    'Audit Duplicate Results - SDE prod': 'audit_duplicate_results',
    'Audit Metrics': 'audit_metrics',
    'Cleaning Assigned To:': 'cleaning_assigned_to',
    'notes': 'notes',
}

In [79]:
# we don't need Collection Title or Migrated? because they have too little information

In [80]:
# check whether we need the Sinequa Config Actual column
df.query('~`Sinequa Config Actual`.isnull() & `Sinequa Config Actual` != `Sinequa Config Guess`')

# turns out we don't

,Collection Title,Migrated?,Name of Resource,Sinequa Config Guess,Sinequa Config Actual,Connector Type,Assignee,URL_original,URL_sinequa,turned_on,...,UrlStayInside,UrlLinkExcluded,UrlDownloadExcluded,UrlFollowExcluded,UrlIndexExcluded,UrlIndexIncluded,IncludedExtensions,ExcludedExtensions,ExcludedFilenames,Cleaned


In [81]:
# fill null configs with empty string for ingesting
df['Sinequa Config Guess'].fillna('', inplace=True)

In [82]:
# do we need connector type? guess not
df['Connector Type'].value_counts()

crawler2    7
Name: Connector Type, dtype: int64

In [83]:
# merge url columns
df["URL_original"].fillna(df["URL_sinequa"], inplace=True)
df["URL_original"].fillna('', inplace=True)

In [84]:
# handle column turned_on
df.turned_on = df.turned_on.fillna(False)
df.turned_on = df.turned_on.replace(1.0, True)
df.turned_on = df.turned_on.replace(0.0, False)
df.turned_on.value_counts()

False    159
True     122
Name: turned_on, dtype: int64

In [85]:
# handle division
df['Division'] = df['Division'].replace('Planetary', 'Planetary Science')

# replace division names with ids
division_mapper = {
    "Planetary Science": 5,
    "Earth Science": 3,
    "Biological and Physical Sciences": 2,
    "Heliophysics": 4,
    "Astrophysics": 1,
}
df['Division'].replace(division_mapper, inplace=True)

In [86]:
# source mapper

source_mapper = {
    "Only in original": 1,
    "both": 2,
    "Only in Sinequa": 3,
}
df['Source'].replace(source_mapper, inplace=True)
df['Source'].value_counts()

2    143
1     81
3     57
Name: Source, dtype: int64

In [87]:
# combine all instruction columns

def generate_notes(row):
    note_fields = [
        'Indexing instructions',
        'Notes',
        'Audit Notes - SDE prod',
        'What Does it Contain?',
        'Audit Comment',
        'Clean-up Changes/Comments by Cleaner',
    ]
    notes = []
    for field in note_fields:
        if not pd.isnull(row[field]):
            notes.append(f'{field}: {row[field]}')
    return '\n\n'.join(notes)

df['notes'] = df.apply(lambda row: generate_notes(row), axis=1)

In [115]:
df[['Name of Resource', 'notes']]

,Name of Resource,notes
0,NAVO Registry,Indexing instructions: Use custom connector bu...
1,Coordinate Calculator,Indexing instructions: Index just this page\n\...
2,Extinction Calculator,Indexing instructions: Index just this page\n\...
3,Velocity Calculator,Indexing instructions: Index just this page\n\...
4,High Level Science Products,Indexing instructions: Index only the content ...
...,...,...
276,PDS Subset Tool Website,
277,PDS Users Guides Website,
278,PDS USGS Pilot Website,
279,PDS Web Chronos Website,


In [88]:
df['Frequency of Indexing/Updates'].replace({'Once a month': 'Monthly', 'Once week': 'Weekly'}, inplace=True)
df['Frequency of Indexing/Updates'].fillna('Weekly', inplace=True)
df['Frequency of Indexing/Updates'].value_counts()

Monthly     182
Weekly       87
Daily         8
Biweekly      4
Name: Frequency of Indexing/Updates, dtype: int64

In [89]:
# frequency mapper

frequency_mapper = {
    "Daily": 1,
    "Weekly": 2,
    "Biweekly": 3,
    "Monthly": 4
}
df['Frequency of Indexing/Updates'].replace(frequency_mapper, inplace=True)
df['Frequency of Indexing/Updates'].value_counts()

4    182
2     87
1      8
3      4
Name: Frequency of Indexing/Updates, dtype: int64

In [90]:
df['Action'].fillna(False, inplace=True)
df['Action'] = df['Action'].replace('Delete', True)
df['Action'].value_counts()

False    251
True      30
Name: Action, dtype: int64

In [91]:
df['Cleaning Assigned To:'].value_counts()

Shravan                                  34
Nothing needed. Deleted as requested.    28
Carson                                   23
Nothing needed. Looks good.              23
Ashish                                   19
Dr. Li                                   16
Looks good. Nothing needed.               1
Looks good. Nothing needed                1
Nothing needed. Looks good                1
Name: Cleaning Assigned To:, dtype: int64

In [92]:
# ignoring this one because we're just starting the cleaning, so it couldn't possibly have been finished.
df['Cleaning Finished? (Yes/No/See Sheets)'].value_counts()

Yes                  55
ignore .xml, .ico     1
Name: Cleaning Finished? (Yes/No/See Sheets), dtype: int64

In [93]:
df['Clean-up Changes/Comments by Cleaner'].value_counts()

- Changed to JSON connector.\n- developed a function plugin to translate values\n- created a new HTML template                                                                                                                                  5
No change                                                                                                                                                                                                                                       3
No Change                                                                                                                                                                                                                                       2
-Added conversion plan SMD_Advnaced_Docs                                                                                                                                                                                                        2
Ok, so I initially created a web

In [94]:
df['Audit Metrics'].value_counts()

Good       68
Update     32
Missing    13
Name: Audit Metrics, dtype: int64

In [95]:
# decided not to bring in information about audit in test
# 1. we're going to reindex anyway
# 2. there's not a lot of information here anyway

print(df['Audit Hierarchy (Matches hierarchy in spreedsheet - Yes / No) - SDE test'].value_counts())
print(df['Audit URL - SDE test'].value_counts())
print(df['Audit Mapping - SDE test'].value_counts())
print(df['Audit Label - SDE test'].value_counts())
print(df['Audit Notes - SDE test'].value_counts())
print(df['Audit Query - SDE test'].value_counts())
print(df['Audit Duplicate Results - SDE test'].value_counts())

Yes    9
No     8
Name: Audit Hierarchy (Matches hierarchy in spreedsheet - Yes / No) - SDE test, dtype: int64
No                                10
Yes                                6
Yes (sort of, content is same)     1
Name: Audit URL - SDE test, dtype: int64
Yes    12
No      5
Name: Audit Mapping - SDE test, dtype: int64
No     9
Yes    7
?      1
Name: Audit Label - SDE test, dtype: int64
with query "LEARN TO USE EARTH OBSERVATIONS", top return is xml file https://sservi.nasa.gov/articles/tag/asteroid-grand-challenge/feed/; with query "We offer in-person and online trainings for beginners and advanced practitioners alike, across a breadth of topics including disaster management, ecological forecasting, agriculture, public health, and air quality and water resources management", 7 duplidate pages are found, which are all correct    1
from "Science Knowledge Sources", there is no docs under "Earth Science Airborne Campaigns" label, only docs under  "Earth Science Suborbital Campaig

In [96]:
# useless
df['Title'].value_counts()

Series([], Name: Title, dtype: int64)

In [97]:
# useless
df['Clean Up Results'].value_counts()

Crawler is successfully geting the paginated results from all planets. We added ignore for .xml and igore for .ico. We updated to use the new treepath. We deleted all content and force reindexed. COME BACK and check that it isn't indexing the search result page itself, just the datapages.    1
Had been turned off. Turned back on and reindexed.                                                                                                                                                                                                                                                   1
had been turned off, turned back on. content deleted and reindexed, took less than a minute.                                                                                                                                                                                                         1
removed images. still need to have longterm fix for titles if desired. wouldn't let me delete content, so I didn't 

In [98]:
# useless
df['Description'].value_counts()

GCMD - CASEI - A Discovery Catalog of NASA Airborne and Field Campaigns                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     5
BPS - The LSDA Website -  Life Sciences Data Archive this contains NASA's Human Research Program (HRP) conducts research and develops technologies that allow humans to travel safely and productively in space.                                                                                                                                                                                                                                                          

In [99]:
# bring this in from the sinequa config
df['Connector'].value_counts()

crawler2      174
json            4
generic         1
hyperindex      1
Name: Connector, dtype: int64

In [100]:
# QC cols are basically useless

print(df['Title QC'].value_counts())
print(df['Title href QC'].value_counts())
print(df['Preview QC'].value_counts())
print(df['Metadata QC'].value_counts())
print(df['Unwanted Docs QC'].value_counts())

Many pages display the word Data, which is what shows on the tab, instead of the actual title    1
Name: Title QC, dtype: int64
ok    1
Name: Title href QC, dtype: int64
Preview was broken. Possibly still being generated    1
ok                                                    1
Name: Preview QC, dtype: int64
ok    1
Name: Metadata QC, dtype: int64
some images were being brought in    1
Name: Unwanted Docs QC, dtype: int64


In [101]:
# we can bring these in from the sinequa configs 
# skip for now

print(df['DynamicUrlList'].value_counts())
print(df['UrlStayInside'].value_counts())
print(df['UrlLinkExcluded'].value_counts())
print(df['UrlDownloadExcluded'].value_counts())
print(df['UrlFollowExcluded'].value_counts())
print(df['UrlIndexExcluded'].value_counts())
print(df['UrlIndexIncluded'].value_counts())
print(df['IncludedExtensions'].value_counts())
print(df['ExcludedExtensions'].value_counts())
print(df['ExcludedFilenames'].value_counts())

SMD_GENELAB_DYNAMIC_URL_LIST                 1
SMD_GENELAB_PUBLICATIONS_DYNAMIC_URL_LIST    1
SMD_TASKBOOK_DYNAMIC_URL_LIST                1
Name: DynamicUrlList, dtype: int64
1.0    190
0.0      7
Name: UrlStayInside, dtype: int64
*/rss/*                                                                                   2
https://pds-imaging.jpl.nasa.gov/data/*                                                   1
*download*                                                                                1
['https://neo.sci.gsfc.nasa.gov/archive/*', 'https://neo.sci.gsfc.nasa.gov/servlet/*']    1
['http://www.ncbi.nlm.nih.gov/*', 'https://taskbook.nasaprs.com/tbp/spaceline.cfm']       1
Name: UrlLinkExcluded, dtype: int64
*/rss/*                                                                                                                                                                                                                                                                         

In [102]:
# useless
df['Cleaned'].value_counts()

Series([], Name: Cleaned, dtype: int64)

In [103]:
new = df[columns_to_bring_in.keys()].copy()
new.rename(columns=columns_to_bring_in, inplace=True)

In [104]:
# do a final fillna on text columns
new.url = new.url.fillna('')
new.url = new.url.replace('Use custom connector built by team', '')
new.url = new.url.replace('See notes', '')
new.config_folder = new.config_folder.fillna('')
new['tree_root'].fillna('', inplace=True)
new['audit_hierarchy'].fillna('', inplace=True)
new['audit_url'].fillna('', inplace=True)
new['audit_mapping'].fillna('', inplace=True)
new['audit_label'].fillna('', inplace=True)
new['audit_query'].fillna('', inplace=True)
new['audit_duplicate_results'].fillna('', inplace=True)
new['audit_metrics'].fillna('', inplace=True)
new['cleaning_assigned_to'].fillna('', inplace=True)

In [105]:
new[~new.url.str.startswith('ht')]

,name,config_folder,url,turned_on,division,source,tree_root,update_frequency,delete,audit_hierarchy,audit_url,audit_mapping,audit_label,audit_query,audit_duplicate_results,audit_metrics,cleaning_assigned_to,notes
0,NAVO Registry,,,False,1,1,,1,False,No,No,No,No,https://sciencediscoveryengine.nasa.gov/app/na...,,,,Indexing instructions: Use custom connector bu...
104,Planetary Data System (PDS),PDS_Planetary_Data_System_(PDS)_Website,,False,5,2,Planetary/Data//Planetary Data System (PDS)/,1,True,,,,,,,Update,Carson,Indexing instructions: Use custom connector bu...
198,PDS API (known as 'Legacy Registry' in the EDP...,PDS_API_Legacy_All,,True,5,2,Planetary Science/Data/Planetary Science Data ...,1,False,No,?,Yes,No,,,Update,Shravan,Indexing instructions: Use custom connector bu...
225,ASTRO Multi Website,ASTRO_Multi_Website,"['https://github.com/adsabs/adsabs-dev-api', '...",False,1,3,Astrophysics/,2,False,,,,,,,,,
235,Helioviewer Documentation Website,Helioviewer_Documentation_Website,"['http://www.jhelioviewer.org/', 'http://swhv....",False,4,3,Heliophysics/Software and Tools/Helioviewer/JH...,2,False,,,,,,,,,
236,LSDA Website Trial,LSDA_Website_Trial,['https://nlsp.nasa.gov/search/?q=all&pagesize...,False,2,3,LSDA/Life Sciences Data Archive/,2,False,,,,,,,,,
237,LSDA Website Trial2,LSDA_Website_Trial2,['https://nlsp.nasa.gov/search/?q=all&pagesize...,False,2,3,LSDA/Life Sciences Data Archive/,2,False,,,,,,,,,
280,TASKBOOK Website,TASKBOOK_Website,,False,5,3,Taskbook,2,False,,,,,,,,,


In [106]:
# remove multi urls
mask = new.url.apply(lambda row: row.startswith('['))
column_name = 'url'
new.loc[mask, column_name] = ''

In [107]:
# which ones don't have URLs
new[~new.url.str.startswith('ht')]

,name,config_folder,url,turned_on,division,source,tree_root,update_frequency,delete,audit_hierarchy,audit_url,audit_mapping,audit_label,audit_query,audit_duplicate_results,audit_metrics,cleaning_assigned_to,notes
0,NAVO Registry,,,False,1,1,,1,False,No,No,No,No,https://sciencediscoveryengine.nasa.gov/app/na...,,,,Indexing instructions: Use custom connector bu...
104,Planetary Data System (PDS),PDS_Planetary_Data_System_(PDS)_Website,,False,5,2,Planetary/Data//Planetary Data System (PDS)/,1,True,,,,,,,Update,Carson,Indexing instructions: Use custom connector bu...
198,PDS API (known as 'Legacy Registry' in the EDP...,PDS_API_Legacy_All,,True,5,2,Planetary Science/Data/Planetary Science Data ...,1,False,No,?,Yes,No,,,Update,Shravan,Indexing instructions: Use custom connector bu...
225,ASTRO Multi Website,ASTRO_Multi_Website,,False,1,3,Astrophysics/,2,False,,,,,,,,,
235,Helioviewer Documentation Website,Helioviewer_Documentation_Website,,False,4,3,Heliophysics/Software and Tools/Helioviewer/JH...,2,False,,,,,,,,,
236,LSDA Website Trial,LSDA_Website_Trial,,False,2,3,LSDA/Life Sciences Data Archive/,2,False,,,,,,,,,
237,LSDA Website Trial2,LSDA_Website_Trial2,,False,2,3,LSDA/Life Sciences Data Archive/,2,False,,,,,,,,,
280,TASKBOOK Website,TASKBOOK_Website,,False,5,3,Taskbook,2,False,,,,,,,,,


In [108]:
def create_fixture(row):
    return {
            "model": "sde_collections.collection",
            "fields": {
                "name": row["name"],
                "config_folder": row["config_folder"],
                "url": row["url"],
                "turned_on": row["turned_on"],
                "division": row["division"],
                "source": row["source"],
                "tree_root": row["tree_root"],
                "update_frequency": row["update_frequency"],
                "delete": row["delete"],
                "audit_hierarchy": row["audit_hierarchy"],
                "audit_url": row["audit_url"],
                "audit_mapping": row["audit_mapping"],
                "audit_label": row["audit_label"],
                "audit_query": row["audit_query"],
                "audit_duplicate_results": row["audit_duplicate_results"],
                "audit_metrics": row["audit_metrics"],
                "cleaning_assigned_to": row["cleaning_assigned_to"],
                "notes": row["notes"],
            }
    }

In [109]:
new['json'] = new.apply(lambda row: create_fixture(row), axis=1)

In [110]:
with open('../sde_collections/fixtures/collections.json', 'w') as fixturefile:
    fixturefile.write(json.dumps(new.json.to_list()))